In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
date_string_format = '%Y/%m/%d %H:%M:%S %p'
date_parserer = lambda s: pd.to_datetime(s,date_string_format)

# read Q2 data:
path = r'C:/Users/yyp/Desktop/Upenn/MUSA/courses/MUSA5080/Final/data/2021_q2.csv'
park = pd.read_csv(
    path, 
    parse_dates=[5,6],
    #date_parser=date_parserer,
    infer_datetime_format=True,
    dayfirst=False,
    cache_dates=True
    )

park = park.loc[park['session_start_dt']<park['session_end_dt']]

# April:
t1 = park.loc[park['Month']==4]
t1 = pd.DataFrame(t1)

# post_id list:
id_list = t1['post_id'].unique()

- function: join_times. 

This function is used to solve the problem of overlapping.

In [ ]:
def join_times(x):
    startdf = pd.DataFrame({'time':x["session_start_dt"], 'what':1})
    enddf = pd.DataFrame({'time':x['session_end_dt'], 'what':-1})
    mergdf = pd.concat([startdf, enddf]).sort_values('time')
    mergdf['running'] = mergdf['what'].cumsum()
    mergdf['newwin'] = mergdf['running'].eq(1) & mergdf['what'].eq(1)
    mergdf['group'] = mergdf['newwin'].cumsum()
    x['group'] = mergdf['group'].loc[mergdf['what'].eq(1)]
    res = x.groupby('group').agg({'session_start_dt':'first','session_end_dt':'last' })
    res["interval"] = (res['session_end_dt']-res['session_start_dt']).dt.seconds
    return res

In [ ]:
p = pd.DataFrame()

for i in id_list:
    r1 = t1.loc[park['post_id']==i]
    r1 = pd.DataFrame(r1)
    p1 = r1.groupby(['Month','post_id']).apply(join_times).reset_index().drop('group', axis=1)
    p = pd.concat([p, p1])

It works fine, but takes way too long time to run. (About 2h)